In [ ]:
pip install XlsxWriter

In [107]:
import pandas as pd
# El objetivo es transformar un CSV de ventas en un DWH simple
df = pd.read_csv("Superstore.csv", encoding='latin1')

In [108]:
print(df.columns)

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')


In [109]:
# Limpieza de datos
columns_to_drop = [
    'Row ID', 'Ship Date', 'Ship Mode', 'Customer Name',
    'Country', 'Postal Code'
]
df = df.drop(columns=columns_to_drop)

In [110]:
# Renombrar columnas
df.rename(columns={
    'Order Date': 'fecha_orden',
    'Order ID': 'orden_id',
    'Customer ID': 'cliente_id',
    'Customer Name': 'nombre_cliente',
    'Segment': 'segmento',
    'Country': 'pais',
    'City': 'ciudad',
    'State': 'estado',
    'Region': 'region',
    'Product ID': 'producto_id',
    'Category': 'categoria',
    'Sub-Category': 'subcategoria',
    'Product Name': 'nombre_producto',
    'Sales': 'ventas',
    'Quantity': 'cantidad',
    'Discount': 'descuento',
    'Profit': 'utilidad'
}, inplace=True)

In [111]:
df['fecha_orden'] = pd.to_datetime(df['fecha_orden'], format='%d-%m-%Y', dayfirst=True)
df['fecha_orden'] = df['fecha_orden'].dt.normalize()
# Ahora extraemos el año, mes y día de la fecha
df['año'] = df['fecha_orden'].dt.year
df['mes'] = df['fecha_orden'].dt.month
df['dia'] = df['fecha_orden'].dt.day

In [105]:
# Creamos las diferentes dimensiones con sus respectivas tablas
dim_cliente = df[['cliente_id', 'segmento']].drop_duplicates()
dim_producto = df[['producto_id', 'categoria', 'subcategoria', 'nombre_producto']].drop_duplicates()
dim_region = df[['region', 'estado', 'ciudad']].drop_duplicates()
dim_region['region_id'] = dim_region.index + 1
df = df.merge(dim_region, on=['region', 'estado', 'ciudad'], how='left')
dim_tiempo = df[['fecha_orden', 'año', 'mes', 'dia']].drop_duplicates()
hechos_ventas = df[['fecha_orden', 'cliente_id', 'producto_id', 'region_id', 'ventas', 'cantidad', 'descuento']]

In [112]:
# Creamos las diferentes dimensiones con sus respectivas tablas
dim_cliente = df[['cliente_id', 'segmento']].drop_duplicates()
dim_producto = df[['producto_id', 'categoria', 'subcategoria', 'nombre_producto']].drop_duplicates()
dim_region = df[['region', 'estado', 'ciudad']].drop_duplicates()
dim_region['region_id'] = dim_region.index + 1
df = df.merge(dim_region, on=['region', 'estado', 'ciudad'], how='left')
dim_tiempo = df[['fecha_orden', 'año', 'mes', 'dia']].drop_duplicates()
dim_tiempo['tiempo_id'] = dim_tiempo.index + 1
df = df.merge(dim_tiempo, on='fecha_orden', how='left')
# Creamos la tabla de hechos
hechos_ventas = df[['tiempo_id', 'cliente_id', 'producto_id', 'region_id', 'ventas', 'cantidad', 'descuento']]

In [114]:
# Crear archivo Excel con varias hojas
with pd.ExcelWriter('ventasdb.xlsx', engine='xlsxwriter') as writer:
    dim_cliente.to_excel(writer, sheet_name='dim_cliente', index=False)
    dim_producto.to_excel(writer, sheet_name='dim_producto', index=False)
    dim_region.to_excel(writer, sheet_name='dim_region', index=False)
    dim_tiempo.to_excel(writer, sheet_name='dim_tiempo', index=False)
    hechos_ventas.to_excel(writer, sheet_name='hechos_ventas', index=False)

print("Archivo 'ventasdb.xlsx' creado con éxito, con todas las tablas.")

Archivo 'ventasdb.xlsx' creado con éxito, con todas las tablas.
